In [1]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing testutility.py


In [3]:
%%writefile file.yaml
file_type: csv
dataset_name: testfile
file_name: test_data
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - Emp ID
    - Name Prefix
    - First Name
    -Middle Initial
    -Last Name
    -Gender
    -E Mail
    -Father's Name
    -Mother's Name
    -Mother's Maiden Name
    -Date of Birth
    -Time of Birth
    -Age in Yrs
    -Weight in Kgs.
    -Date of Joining
    -Quarter of Joining
    -Half of Joining
    -Year of Joining
    -Month of Joining
    -Month Name of Joining
    -Short Month
    -Day of Joining
    -DOW of Joining
    -Short DOW
    -Age in Company (Years)
    -Salary
    -Last % Hike
    -SSN
    -Phone No. 
    -Place Name
    -County
    -City
    -State
    -Zip
    -Region
    -User Name
    -Password

Overwriting file.yaml


In [1]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

ERROR:root:while scanning a simple key
  in "file.yaml", line 12, column 5
could not find expected ':'
  in "file.yaml", line 13, column 5


In [3]:
#inspecting data of config file
config_data

In [4]:
# Normal reading process of the file
import pandas as pd
df_sample = pd.read_csv("HR2m.csv",delimiter=',')
df_sample.head()

,Emp ID,Name Prefix,First Name,Middle Initial,Last Name,Gender,E Mail,Father's Name,Mother's Name,Mother's Maiden Name,...,SSN,Phone No.,Place Name,County,City,State,Zip,Region,User Name,Password
0,548811,Mr.,Owen,E,Riebe,M,owen.riebe@bp.com,Wilford Riebe,Codi Riebe,Seely,...,119-98-3586,231-362-2811,Freeland,Saginaw,Freeland,MI,48623,Midwest,oeriebe,w8m|6|dIy1;
1,931402,Mr.,Derrick,K,Selvage,M,derrick.selvage@hotmail.com,Theo Selvage,Karry Selvage,Osby,...,317-35-9195,209-385-6373,Pasadena,Los Angeles,Pasadena,CA,91125,West,dkselvage,cPm5z?uN]I#FxfD
2,254493,Mr.,Dwain,B,Obrien,M,dwain.obrien@gmail.com,James Obrien,Enriqueta Obrien,Kraemer,...,487-29-2725,339-773-9763,Brockton,Plymouth,Brockton,MA,2303,Northeast,dbobrien,jn}KaXnd
3,372623,Ms.,Ilda,S,Mcmaster,F,ilda.mcmaster@microsoft.com,Rey Mcmaster,Vada Mcmaster,Olmstead,...,503-45-6930,304-529-0734,Falls Mill,Falls Mill,Falls Mill,WV,26620,South,ismcmaster,VI/6AuNO
4,144917,Drs.,Paulina,N,Bump,F,paulina.bump@yahoo.com,Stanford Bump,Myung Bump,Weekes,...,021-94-1781,385-280-9502,Goshen,Utah,Goshen,UT,84633,West,pnbump,k&CZjitR>]


In [9]:
pd.read_csv("./HR2m.csv")

,Emp ID,Name Prefix,First Name,Middle Initial,Last Name,Gender,E Mail,Father's Name,Mother's Name,Mother's Maiden Name,...,SSN,Phone No.,Place Name,County,City,State,Zip,Region,User Name,Password
0,548811,Mr.,Owen,E,Riebe,M,owen.riebe@bp.com,Wilford Riebe,Codi Riebe,Seely,...,119-98-3586,231-362-2811,Freeland,Saginaw,Freeland,MI,48623,Midwest,oeriebe,w8m|6|dIy1;
1,931402,Mr.,Derrick,K,Selvage,M,derrick.selvage@hotmail.com,Theo Selvage,Karry Selvage,Osby,...,317-35-9195,209-385-6373,Pasadena,Los Angeles,Pasadena,CA,91125,West,dkselvage,cPm5z?uN]I#FxfD
2,254493,Mr.,Dwain,B,Obrien,M,dwain.obrien@gmail.com,James Obrien,Enriqueta Obrien,Kraemer,...,487-29-2725,339-773-9763,Brockton,Plymouth,Brockton,MA,2303,Northeast,dbobrien,jn}KaXnd
3,372623,Ms.,Ilda,S,Mcmaster,F,ilda.mcmaster@microsoft.com,Rey Mcmaster,Vada Mcmaster,Olmstead,...,503-45-6930,304-529-0734,Falls Mill,Falls Mill,Falls Mill,WV,26620,South,ismcmaster,VI/6AuNO
4,144917,Drs.,Paulina,N,Bump,F,paulina.bump@yahoo.com,Stanford Bump,Myung Bump,Weekes,...,021-94-1781,385-280-9502,Goshen,Utah,Goshen,UT,84633,West,pnbump,k&CZjitR>]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,761314,Mr.,Howard,D,Hoye,M,howard.hoye@earthlink.net,Rueben Hoye,Shemika Hoye,Almeida,...,048-15-4232,240-988-9724,Upper Marlboro,Prince George's,Upper Marlboro,MD,20773,South,hdhoye,ta[7S&#_V%*{
1048571,662492,Mrs.,Daniella,Y,Plumlee,F,daniella.plumlee@aol.com,Abdul Plumlee,Jade Plumlee,Strom,...,326-11-5713,216-876-2993,Wickliffe,Lake,Wickliffe,OH,44092,Midwest,dyplumlee,V+m;I%<%t<xZvG^
1048572,510987,Mrs.,Yolando,D,Brucker,F,yolando.brucker@shaw.ca,Edgardo Brucker,Sharyn Brucker,Olivarria,...,396-33-7753,210-224-0692,Tennessee Colony,Anderson,Tennessee Colony,TX,75884,South,ydbrucker,PrDn%<u3h
1048573,476082,Mrs.,Lynda,R,Villacorta,F,lynda.villacorta@hotmail.com,Ronny Villacorta,Miss Villacorta,Lanham,...,101-02-9543,212-472-8735,Warsaw,Wyoming,Warsaw,NY,14569,Northeast,lrvillacorta,5C}AZPzJZ]q8}


In [10]:
df

NameError: name 'df' is not defined